In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tensorflow.keras.models import load_model

In [2]:
dataset_path = './dataset'

if not os.path.exists(dataset_path):
  raise BaseException("Path not found")

train_data = pd.read_csv(os.path.join(dataset_path, "Train.csv"))
train_data

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png
...,...,...,...,...,...,...,...,...
39204,52,56,5,6,47,51,42,Train/42/00042_00007_00025.png
39205,56,58,5,5,51,53,42,Train/42/00042_00007_00026.png
39206,58,62,5,6,53,57,42,Train/42/00042_00007_00027.png
39207,63,69,5,7,58,63,42,Train/42/00042_00007_00028.png


In [3]:
test_data = pd.read_csv(os.path.join(dataset_path, "Test.csv"))
test_data

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,53,54,6,5,48,49,16,Test/00000.png
1,42,45,5,5,36,40,1,Test/00001.png
2,48,52,6,6,43,47,38,Test/00002.png
3,27,29,5,5,22,24,33,Test/00003.png
4,60,57,5,5,55,52,11,Test/00004.png
...,...,...,...,...,...,...,...,...
12625,42,41,5,6,37,36,12,Test/12625.png
12626,50,51,6,5,45,46,33,Test/12626.png
12627,29,29,6,6,24,24,6,Test/12627.png
12628,48,49,5,6,43,44,7,Test/12628.png


In [4]:
def load_image(row):
  try:
    path = os.path.join(dataset_path, row["Path"])
    with Image.open(path) as img:
      cropped_img = img.crop((row['Roi.X1'], row['Roi.Y1'], row['Roi.X2'], row['Roi.Y2']))
      resized_img = np.array(cropped_img.resize((32, 32), resample=Image.Resampling.LANCZOS))
      return resized_img / 255.0
  except Exception as e:
    print(f"Error loading image {row['Path']}: {e}")
    return None

In [5]:
train_data['image'] = train_data.apply(load_image, axis=1)
test_data['image'] = test_data.apply(load_image, axis=1)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

2024-12-23 01:19:27.449264: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-23 01:19:27.461387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734909567.471304    4313 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734909567.474020    4313 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-23 01:19:27.484291: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [7]:
num_classes = train_data['ClassId'].nunique()

x_train = np.stack(train_data['image'].values)
y_train = to_categorical(train_data['ClassId'].values, num_classes)
print(f"[Training] Feature shape: {x_train.shape}, Label shape: {y_train.shape}")

x_test = np.stack(test_data['image'].values)
y_test = to_categorical(test_data['ClassId'].values, num_classes)
print(f"[Testing] Feature shape: {x_test.shape}, Label shape: {y_test.shape}")

[Training] Feature shape: (39209, 32, 32, 3), Label shape: (39209, 43)
[Testing] Feature shape: (12630, 32, 32, 3), Label shape: (12630, 43)


In [8]:
def build_model(hp):    
  hp_filters = hp.Int('filters', min_value=32, max_value=128, step=32)
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
  hp_num_conv_layers = hp.Int('hp_num_additional_conv_layers', min_value=0, max_value=2)
  hp_num_dense_layers = hp.Int('num_dense_layers', min_value=0, max_value=3)
  
  model = Sequential()

  model.add(Conv2D(hp_filters, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D((2, 2)))
  
  i = 2
  for _ in range(hp_num_conv_layers):
    model.add(Conv2D(hp_filters * i, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    i = i * 2

  model.add(GlobalAveragePooling2D())

  for _ in range(hp_num_dense_layers):
      model.add(Dense(hp_units, activation='relu'))
      model.add(BatchNormalization())
      model.add(Dropout(0.5))

  model.add(Dense(43, activation='softmax'))

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='tmp',
                     project_name='signs-recognition')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=50, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

I0000 00:00:1734909577.569782    4313 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6847 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


Reloading Tuner from tmp/signs-recognition/tuner0.json


In [9]:
print("Best parameters:")
print(f"filters={best_hps.get('filters')}")
print(f"units={best_hps.get('units')}")
print(f"learning_rate={best_hps.get('learning_rate')}")
print(f"hp_num_additional_conv_layers={best_hps.get('hp_num_additional_conv_layers')}")
print(f"num_dense_layers={best_hps.get('num_dense_layers')}")

Best parameters:
filters=64
units=480
learning_rate=0.001
hp_num_additional_conv_layers=2
num_dense_layers=2


In [11]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), callbacks=[stop_early])

Epoch 1/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.4461 - loss: 2.3580 - val_accuracy: 0.9277 - val_loss: 0.2400
Epoch 2/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9639 - loss: 0.1217 - val_accuracy: 0.8971 - val_loss: 0.4246
Epoch 3/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9778 - loss: 0.0788 - val_accuracy: 0.9502 - val_loss: 0.1700
Epoch 4/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9802 - loss: 0.0667 - val_accuracy: 0.9534 - val_loss: 0.1793
Epoch 5/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9849 - loss: 0.0492 - val_accuracy: 0.9393 - val_loss: 0.2556
Epoch 6/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9855 - loss: 0.0463 - val_accuracy: 0.9637 - val_loss: 0.1616
Epoch 7/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9872 - loss: 0.0429 - val_accuracy: 0.9686 - val_loss: 0.1587
Epoch 8/100
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9889 - loss:

In [12]:
model.evaluate(x_test, y_test)

395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9719 - loss: 0.1710


[0.17871561646461487, 0.970546305179596]

In [12]:
model.save("best_model.keras")

In [ ]:
model = load_model("best_model.keras")
model.evaluate(x_test, y_test)